# Writting the package documentation

- tools
- hosting platforms
- what makes a good doc (quickstart, install, user guide, example gallery, API reference, contributing guide...)
- configure and build our doc with Sphinx
- Use auto-API for the API reference